In [1]:
import numpy as np
import time
import pickle
import os
import matplotlib.pyplot as plt
import warnings
import pandas as pd

warnings.filterwarnings('ignore')

In [2]:
print("----------------Reading the Data-------------------------")
PATH = os.getcwd()
os.chdir('Alphabets/')

X_train = pd.read_csv('train.csv', sep=',', header=None, index_col=False)
X_test = pd.read_csv('test.csv', sep=',', header=None, index_col=False)

train_class = X_train[X_train.columns[-1]]
test_actual_class = X_test[X_test.columns[-1]]

X_train = X_train.drop(X_train.columns[-1], axis=1)
X_test = X_test.drop(X_test.columns[-1], axis=1)

print("----------------Data Reading completed-------------------")

os.chdir('../')

X_train = X_train/255
X_test = X_test/255
m = X_train.shape[0] # Number of Training Samples
n = X_train.shape[1] # Number of input features

print("The total number of training samples = {}".format(m))
print("The number of features = {}".format(n))

----------------Reading the Data-------------------------
----------------Data Reading completed-------------------
The total number of training samples = 13000
The number of features = 784


In [3]:
#To get the one hot encoding of each label
print("--------Perform 1-hot encoding of class labels------------")

train_class_enc = pd.get_dummies(train_class).to_numpy()
test_actual_class_enc = pd.get_dummies(test_actual_class).to_numpy()

--------Perform 1-hot encoding of class labels------------


In [4]:
#Add the intercept term to the data samples both in training and test dataset
X_train = np.hstack((np.ones((m,1)),X_train.to_numpy()))
X_test = np.hstack((np.ones((X_test.shape[0],1)),X_test.to_numpy()))

In [9]:
lr = 0.1
arch = [10] #means one hidden layer with 2 perceptrons 
M = 100 # Mini-Batch Size
r = np.max(train_class) + 1 # Default value of the number of classes = 26

In [10]:
theta1 = np.random.random((n+1,arch[0]))
theta2 = np.random.random((arch[0],r))

In [11]:
def activation(x):
    return 1/(1+np.exp(-x))

In [ ]:
for it in range(60000):
    # Forward Propagation
    l0 = X_train
    l1 = activation(np.dot(l0, theta1))
    l2 = activation(np.dot(l1, theta2))

    if (it % 5000 == 0):
        print("Error = "+str(np.mean(np.abs(train_class_enc-l2))))

    #Backward Propagation
    delta_l2 = (1/m)*(train_class_enc - l2)*l2*(1-l2)

    delta_l1 = np.dot(delta_l2, theta2.T)*l1*(1-l1)

    theta1 += lr*np.dot(l0.T, delta_l1)
    theta2 += lr*np.dot(l1.T, delta_l2)

Error = 0.9520288128047689
Error = 0.0739838002926372
Error = 0.0739645176161679


In [214]:
def forward_prop(data, theta):
    l0 = data
    l1 = activation(np.dot(l0, theta[0]))
    l2 = activation(np.dot(l1, theta[1]))
    return l2

In [215]:
test_pred_class = forward_prop(X_test, [theta1, theta2])
test_acc = 0
for i in range(len(test_actual_class_enc)):
    if (np.array_equal(test_pred_class[i], test_actual_class_enc[i])):
        test_acc++
test_acc /= X_test.shape[0]

print("The Test Accuracy of the model = {}%".format(test_acc*100))

array([ True,  True, False,  True, False,  True,  True,  True,  True,
        True,  True,  True,  True, False,  True,  True, False,  True,
        True,  True,  True,  True, False,  True, False,  True])

In [ ]:
#Mini-Batch

batch_size = M
mini_batch = [(X_train[i:i+r,:], train_class_enc[i:i+r]) for i in range(0, m, M)]